In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 检查GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 生成一些随机数据
X = torch.randn(100, 1).to(device)
true_w = 2.0
true_b = 1.0
y = (true_w * X + true_b + 0.1 * torch.randn(100, 1)).to(device)

# 定义线性神经网络
model = nn.Linear(1, 1).to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 训练
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 40 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 查看训练后的参数
w = model.weight.item()
b = model.bias.item()
print(f"Learned parameters: w={w:.4f}, b={b:.4f}")

ModuleNotFoundError: No module named 'torch'